In [ ]:
import torch
import os
import cv2
import zipfile
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('')
zip_ref.close()

--2022-07-27 14:03:00--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.72.44.106, 2600:1417:3f:8a0::e59, 2600:1417:3f:898::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.72.44.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M   273MB/s    in 2.9s    

2022-07-27 14:03:04 (273 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



In [ ]:
Flag = True
class make_data():
    img_size = 50
    cats = "PetImages/Cat"
    dogs = "PetImages/Dog"
    labels = {cats: 0, dogs: 1}
    training_data = []

    num_cat = 0
    num_dog = 0

    def resize_data(self):
        for label in self.labels:
            for file in tqdm(os.listdir(label)):
                if "jpg" in file:
                    try:
                        path = os.path.join(label, file)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.img_size, self.img_size))
                        self.training_data.append([np.array(img), np.eye(2)[self.labels[label]]])

                        if label == self.cats:
                            self.num_cat += 1
                        elif label == self.dogs:
                            self.num_dog += 1

                    except Exception as e:
                        pass

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)

if Flag:
    data = make_data()
    data.resize_data()
    print('Cats:',data.num_cat)
    print('Dogs:',data.num_dog)
training_data = np.load("training_data.npy", allow_pickle=True)

100%|██████████| 12501/12501 [00:13<00:00, 952.91it/s]
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Cats: 12476
Dogs: 12470


In [ ]:
device = "cuda"
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5) 
        self.pool1 = nn.MaxPool2d((2, 2))
        self.pool2 = nn.MaxPool2d((2, 2))
        self.pool3 = nn.MaxPool2d((2, 2))
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 2)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.flatten(start_dim=1) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)
loss_function = nn.CrossEntropyLoss()

X = (torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50))/250
y = torch.Tensor([i[1] for i in training_data])
val_size = int(len(X)*0.1)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

def train(net):
    batch_size = 20
    epochs = 20
    for epoch in range(epochs):
        for i in tqdm(range(0, len(train_X), batch_size)):
            batch_X = train_X[i:i+batch_size].view(-1,1,50,50)
            batch_y = train_y[i:i+batch_size]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            outputs = net(batch_X)
            loss = loss_function(outputs, torch.max(batch_y, 1)[1])
            loss.backward()
            optimizer.step()
        print(f"Epoch: {epoch}. Loss: {loss}")

def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy:", round(correct/total,3))
train(net)
test(net)

100%|██████████| 1123/1123 [00:03<00:00, 292.52it/s]


Epoch: 0. Loss: 0.3145280182361603


100%|██████████| 1123/1123 [00:03<00:00, 295.73it/s]


Epoch: 1. Loss: 0.3236575722694397


100%|██████████| 1123/1123 [00:03<00:00, 297.87it/s]


...

Epoch: 18. Loss: 0.31334054470062256


100%|██████████| 1123/1123 [00:03<00:00, 294.62it/s]


Epoch: 19. Loss: 0.31438425183296204


100%|██████████| 2494/2494 [00:01<00:00, 1437.45it/s]

Accuracy: 0.735
